In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import mlb
import gc
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
main_path = r"../input/mlb-player-digital-engagement-forecasting"
preload_path = r"../input/k/ssmohanty/mlb-master-ads-creation-v1-basic-eda"

#---------------------------------------------------------------------------
master_ads = pd.read_csv(os.path.join(preload_path, "master_ads_4_left_join.csv"))
master_ads.drop(columns=['Unnamed: 0','numberOfFollowers_x','numberOfFollowers_y'],inplace=True)
print(master_ads.shape)
print(master_ads.info())
master_ads.head(3)

In [ ]:
# master_ads['trace_back_flag'] = np.where(master_ads.notna().all(axis=1), 1, 0)
master_ads = master_ads.fillna(0)
master_ads.info()

In [ ]:
master_ads.isnull().sum()

In [ ]:
# master_ads[master_ads['trace_back_flag']==1].head()

In [ ]:
dateCols = [col for col in master_ads.columns if ('date' in col) | ('Date' in col)]
dateCols

In [ ]:
master_ads[['playerId','eng_date_pre','eng_date']].tail()

In [ ]:
master_ads[['target1','target2','target3','target4']].describe()

In [ ]:
target_1_med = master_ads['target1'].mean()
target_2_med = master_ads['target2'].mean()
target_3_med = master_ads['target3'].mean()
target_4_med = master_ads['target4'].mean()

## Necessary libraries for ML pred

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb

## Train-Test Split

In [ ]:
target_cols = ['target1','target2','target3','target4']
total_cols = list(master_ads.columns)
X_cols = [col for col in total_cols if col not in target_cols]

print('Total Cols :',len(total_cols))
print('X Cols :',len(X_cols))
#--------------------------------------------------------------------------------------------

X = master_ads.loc[:, X_cols]
y = master_ads.loc[:, target_cols]

#------------------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.03,test_size=0.15, random_state=100)

print('Train X shape :',X_train.shape)
print('Test X shape :',X_test.shape)

## Check for un-encoded categorical cols, date columns, Id columns

In [ ]:
catCols = [col for col in X_train.columns if X_train[col].dtype=="O"]
len(catCols)
catCols

dateCols = [col for col in X_train.columns if ('date' in col) | ('Date' in col)]
len(dateCols)

IdCols = [col for col in X_train.columns if ('Id' in col) | ('id' in col)]
len(IdCols)
IdCols

print(dateCols)
print(IdCols)
print(catCols)

## Further data processing (Removing Level Columns from train and test X)

In [ ]:
level_cols = ['eng_date','eng_date_pre','playerId']

X_train_1 = X_train.drop(columns=level_cols)
X_test_1 = X_test.drop(columns=level_cols)

#--------------------------------------------
print('Train X reduced shape :',X_train_1.shape)
print('Test X reduced shape :',X_test_1.shape)

In [ ]:
#X_train_1 = X_train_1.sample(100000,random_state=100)


#print('Train shrinked shape :',X_train_1.shape)

In [ ]:
X_train_1.info()
X_test_1.info()

In [ ]:
del X_train
del X_test
#del master_ads

gc.collect()

In [ ]:
# ['awardPlayerTeamId', 'awardSeason', 'fromTeamId', 'toTeamId']
X_train_1.drop(columns=['awardPlayerTeamId', 'awardSeason', 'fromTeamId', 'toTeamId'],inplace=True)
X_test_1.drop(columns=['awardPlayerTeamId', 'awardSeason', 'fromTeamId', 'toTeamId'],inplace=True)

In [ ]:
X_train_1.isnull().sum()

## XGB Regressor w/ multiple o/p's (sklearn wrapper)

In [ ]:
xgb.XGBRegressor(random_state=100).get_params()

In [ ]:
#from xgboost import xgb

#--------------------------------------------------
estimator = xgb.XGBRegressor(reg_alpha = 50, reg_almbda = 60,random_state=100,n_jobs=-1)
model = MultiOutputRegressor(estimator, n_jobs=-1)

#--------------------------------------------------
model.fit(X_train_1, y_train)

#--------------------------------------------------
pred_xgb_multi = model.predict(X_test_1)
               

#--------------------------------------------------
pred_xgb_multi.shape 

In [ ]:
def scoring(truth_array,pred_array):
    
    diff = abs(truth_array - pred_array)
    
    print(diff[0:3])
    
    mean_col_wise = np.mean(diff,axis=0)
    print(mean_col_wise)
    
    mean_MAE = np.mean(mean_col_wise)
    
    return mean_MAE

In [ ]:
np.array(y_test)[0:3]

In [ ]:
pred_xgb_multi[0:3]

In [ ]:
xgb_score = scoring(np.array(y_test),pred_xgb_multi)
xgb_score

In [ ]:
example_test = pd.read_csv(os.path.join(main_path, "example_test.csv"))
example_test.head(3)

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

def data_extractor(df_,col_):
    
    print('------- Data Extraction for :',col_,'-------')
    
    final_df = pd.DataFrame()
    
    tmp = df_[col_]
    tmp = tmp.dropna()
    
    #-------------------------------------------------------
    for i in range(len(tmp)):
        tmpdf = unpack_json(tmp.iloc[i])
        final_df = final_df.append(tmpdf)
    
    #-------------------------------------------------------
    print('Shape of final df BEFORE:',final_df.shape)
    final_df = final_df.drop_duplicates()
    print('Shape of final df AFTER:',final_df.shape)
    print(final_df.columns)
    #-------------------------------------------------------
    
    return final_df


def label_encoding_test(unique,col,df):
    
    print('-----',col,'-----')
    
    print('Before :',len(unique))
    encodes_pre = unique
    
    encodes = np.arange(len(encodes_pre))
    encodes = list(encodes+1)

    mapping = dict(zip(encodes_pre,encodes))
    df = df.replace({col:mapping})
    print('After :',df[col].nunique())
    
    return df

def date_type_converter(df):
    
    date_cols = [col for col in list(df.columns) if (('date' in col) | ('Date' in col))]
    
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
        
    return df

In [ ]:
example_test.columns

In [ ]:
example_test.isnull().sum()

In [ ]:
test_df = data_extractor(example_test,'teamTwitterFollowers')
test_df.head()

In [ ]:
# transactions_features_ = ['playerId', 'date', 'fromTeamId', 'toTeamId','typeCode','typeDesc']
# team_twitter_features = ['date', 'teamId','twitterHandle',
#        'numberOfFollowers']#twitterHandle_x will be of team

#['awardPlayerTeamId', 'awardSeason', 'fromTeamId', 'toTeamId']

standings_features = ['season', 'gameDate', 'divisionId', 'teamId', 'streakCode',
       'divisionRank', 'leagueRank', 'wildCardRank', 'leagueGamesBack',
       'sportGamesBack', 'divisionGamesBack', 'wins', 'losses', 'pct',
       'runsAllowed', 'divisionChamp', 'divisionLeader',
       'wildCardLeader', 'eliminationNumber', 'wildCardEliminationNumber',
       'homeWins', 'homeLosses', 'awayWins', 'awayLosses', 'lastTenWins',
       'lastTenLosses', 'extraInningWins', 'extraInningLosses', 'oneRunWins',
       'oneRunLosses', 'dayWins', 'dayLosses', 'nightWins', 'nightLosses',
       'grassWins', 'grassLosses', 'turfWins', 'turfLosses', 'divWins',
       'divLosses', 'alWins', 'alLosses', 'nlWins', 'nlLosses', 'xWinLossPct']
# player_twitter_features = ['date', 'playerId', 'twitterHandle',
#        'numberOfFollowers'] #twitterHandle_y will be of player
player_scores_features = ['home', 'gamePk', 'gameDate', 'teamId',
       'playerId', 'jerseyNum', 'positionCode',
       'positionType', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']
# awards_df_features = ['awardId', 'awardDate', 'awardSeason', 'playerId', 'awardPlayerTeamId']

In [ ]:

final_features = list(X_train_1.columns)

#--------------------------------------------------------------------------------------------------------------------
def ads_generator(df,final_features_generic):
    
    to_extract = ['standings','playerBoxScores']
    
    start = time.time()
    
    df_final = data_extractor(df,'playerBoxScores')
    df_final = df_final[player_scores_features]
    df_final = date_type_converter(df_final)
    
    to_extract.remove('playerBoxScores')
    
    for col in to_extract:
        
        df_temp = data_extractor(df,col)
        
#         if col == 'transactions':
#             qc_df = df_temp.copy()
        
        df_temp = date_type_converter(df_temp)
        
        if 'date' in list(df_temp.columns):
            print('1')
        
        
        if col == 'transactions':
            print('---------------------- Merging Operation for :',col,'------------------------')
            df_final = pd.merge(df_final,df_temp[transactions_features_],
                                left_on=['playerId','gameDate'],
                                right_on=['playerId','date'],how='left')
        
        elif col == 'teamTwitterFollowers':
            print('---------------------- Merging Operation for :',col,'------------------------')
            df_final = pd.merge(df_final,df_temp[team_twitter_features],
                                    left_on=['teamId','gameDate'],
                                    right_on=['teamId','date'],how='left')
        elif col == 'standings':
            print('---------------------- Merging Operation for :',col,'------------------------')
            df_final = pd.merge(df_final,df_temp[standings_features],
                                    left_on=['teamId','gameDate'],
                                    right_on=['teamId','gameDate'],how='left')
            
        elif col == 'playerTwitterFollowers':
            print('---------------------- Merging Operation for :',col,'------------------------')
            df_final = pd.merge(df_final,df_temp[player_twitter_features],left_on=['playerId','gameDate'],
                                right_on=['playerId','date'],how='left')
        
        elif col == 'awards_df':
            print('---------------------- Merging Operation for :',col,'------------------------')
            df_final = pd.merge(df_final,df_temp[awards_df_features],left_on=['playerId','gameDate'],
                                right_on=['playerId','awardDate'],how='left')
            
    df_final = df_final.fillna(0)
    
    #----------------------------------------------------------------------------------------------------
#     df_final = label_encoding_test(twitterHandle_x_unique,'twitterHandle_x',df_final)
#     df_final = label_encoding_test(awards_df_unique,'awards_df',df_final)
#     df_final = label_encoding_test(twitterHandle_y_unique,'twitterHandle_y',df_final)
    
    #----------------------------------------------------------------------------------------------------
    level_cols_test = ['gameDate']
    df_final_1 = df_final.drop(columns=level_cols_test)
    
    df_final_1 = df_final_1[final_features_generic]
    
    end = time.time()
    print('Time to Run:',end-start)
    
    return df_final


In [ ]:
pd.DataFrame(example_test.iloc[0]).T

In [ ]:
pd.DataFrame(example_test.iloc[0]).T['date']

In [ ]:
#example_test
test_ads_gen_df = ads_generator(pd.DataFrame(example_test.iloc[0]).T,final_features)

In [ ]:
test_ads_gen_df = test_ads_gen_df[list(X_train_1.columns)]
type(test_ads_gen_df)

In [ ]:
test_ads_gen_df.head()

In [ ]:
X_test_1.head(2)

In [ ]:
test_preds = model.predict(test_ads_gen_df)
test_preds

In [ ]:
test_preds[:,0].shape

In [ ]:
example_submission = pd.read_csv(os.path.join(main_path, "example_sample_submission.csv"))
example_submission.head(3)

In [ ]:
print(example_submission[0:2])

In [ ]:
players = pd.read_csv(os.path.join(main_path, "players.csv"))
players.head(3)

In [ ]:
# type(players[players['playerId']>50000000000])

In [ ]:
# x/hsybdksh

In [ ]:
# try:
#     x/hsybdksh
# except:
#     ax = 897
    
# print(ax)

## Submitting Predictions

In [ ]:
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

In [ ]:
# Until the end

counter = 1

for (test_df, sample_prediction_df) in iter_test:
    
    sample_prediction_df_copy = sample_prediction_df.copy()
    
    print('------------------------------------ Starting Date No:',counter,' -----------------------------------------')
    
    print(sample_prediction_df_copy[0:2])
    print('-------------------------')
        
    sample_prediction_df_copy = sample_prediction_df_copy.reset_index(drop=True)
    
    print(sample_prediction_df_copy[0:2])
    
    # creat dataset
    sample_prediction_df_copy['playerId'] = sample_prediction_df_copy['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    
    print('Sample Pred Shape :',sample_prediction_df_copy.shape)
    
    try:
        # Example: unpack a dataframe from a json column
        test_ads_ext = ads_generator(test_df,final_features)
        test_ads_ext = test_ads_ext.drop_duplicates(subset=['playerId'])
        print('Pre-Merging of extracted ADS Shape :',test_ads_ext.shape)
        
    except:
        print('Running Alternative Route 1')
        test_ads_ext = 0
        
    
    players_curr_day = sample_prediction_df_copy[['playerId']].drop_duplicates()
    print('Pre-Merging of players Shape :',players_curr_day.shape)

    #------------------------------------------------------------------------------------------------
    if type(test_ads_ext)!=int:
        
        try:

            test_ads_ext_final = pd.merge(players_curr_day,test_ads_ext,on=['playerId'],how='left')
            print('Post Merging final ADS Shape :',test_ads_ext_final.shape)

            test_ads_ext_final = test_ads_ext_final.fillna(0)
            print('Post Null Treatment ADS Shape :',test_ads_ext_final.shape)

            test_ads_ext_final = test_ads_ext_final[final_features]
            print('Shape of prediction X :',test_ads_ext_final.shape)

            pred = model.predict(test_ads_ext_final)

            sample_prediction_df['target1'] = np.clip(pred[:,0], 0, 100)
            sample_prediction_df['target2'] = np.clip(pred[:,1], 0, 100)
            sample_prediction_df['target3'] = np.clip(pred[:,2], 0, 100)
            sample_prediction_df['target4'] = np.clip(pred[:,3], 0, 100)

            sample_prediction_df = sample_prediction_df.fillna(0.)
        
        except:
            print('Running Alternative Route 2')
            # Make your predictions for the next day's engagement
            sample_prediction_df["target1"] = target_1_med
            sample_prediction_df["target2"] = target_2_med
            sample_prediction_df["target3"] = target_3_med
            sample_prediction_df["target4"] = target_4_med
            
        
    elif test_ads_ext == 0:
        print('Running Alternative Route 3')
        # Make your predictions for the next day's engagement
        sample_prediction_df["target1"] = target_1_med
        sample_prediction_df["target2"] = target_2_med
        sample_prediction_df["target3"] = target_3_med
        sample_prediction_df["target4"] = target_4_med
          
    #test_ads_ext_final = test_ads_ext_final.drop_duplicates() #Comment out - suspected
    
    #print('Post Merging & dupl drop final ADS Shape :',test_ads_ext_final.shape)
    
    
    # Submit your predictions 
    env.predict(sample_prediction_df)

In [ ]:
# # Until the end

# for (test_df, sample_prediction_df) in iter_test:
    
#         # Example: unpack a dataframe from a json column
#         #today_games = unpack_json(test_df['games'].iloc[0])
    
#         # Make your predictions for the next day's engagement
#         sample_prediction_df["target1"] = 0.26764763
#         sample_prediction_df["target2"] = 0.4
#         sample_prediction_df["target3"] = 0.5
#         sample_prediction_df["target4"] = 0.9
        
#         sample_prediction_df["target1"] = sample_prediction_df["target1"].astype('float32')
#         sample_prediction_df["target2"] = sample_prediction_df["target2"].astype('float32')
#         sample_prediction_df["target3"] = sample_prediction_df["target3"].astype('float32')
#         sample_prediction_df["target4"] = sample_prediction_df["target4"].astype('float32')
        
    
#         # Submit your predictions 
#         env.predict(sample_prediction_df)

In [ ]:
# print(sample_prediction_df.dtypes)
# sample_prediction_df.head()


In [ ]:

# counter = 1

# #------------------------------------------------------------------------------------------------------------------------
# for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
#     sample_prediction_df_copy = sample_prediction_df.copy()
    
#     print('------------------------------------ Starting Date No:',counter,' -----------------------------------------')
    
#     print(sample_prediction_df[0:2])
#     print('-------------------------')
    
#     sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    
#     print(sample_prediction_df[0:2])
    
#     # creat dataset
#     sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
#                                         .map(lambda x: int(x.split('_')[1]))
    
#     print('Sample Pred Shape :',sample_prediction_df.shape)
    
#     test_ads_ext = ads_generator(test_df,final_features)
#     test_ads_ext = test_ads_ext.drop_duplicates(subset=['playerId'])
    
#     players_curr_day = sample_prediction_df[['playerId']].drop_duplicates()
    
#     print('Pre-Merging of players Shape :',players_curr_day.shape)
#     print('Pre-Merging of extracted ADS Shape :',test_ads_ext.shape)
    
#     test_ads_ext_final = pd.merge(players_curr_day,test_ads_ext,on=['playerId'],how='left')
          
#     print('Post Merging final ADS Shape :',test_ads_ext_final.shape)
          
#     test_ads_ext_final = test_ads_ext_final.drop_duplicates()
    
#     print('Post Merging & dupl drop final ADS Shape :',test_ads_ext_final.shape)
    
#     test_ads_ext_final = test_ads_ext_final.fillna(0)
          
#     print('Post Null Treatment ADS Shape :',test_ads_ext_final.shape)
    
#     test_ads_ext_final = test_ads_ext_final[final_features]
#     print('Shape of prediction X :',test_ads_ext_final.shape)
    
#     pred = model.predict(test_ads_ext_final)
    
#     sample_prediction_df['target1'] = np.clip(pred[:,0], 0, 100)
#     sample_prediction_df['target2'] = np.clip(pred[:,1], 0, 100)
#     sample_prediction_df['target3'] = np.clip(pred[:,2], 0, 100)
#     sample_prediction_df['target4'] = np.clip(pred[:,3], 0, 100)
    
#     sample_prediction_df = sample_prediction_df.fillna(0.)
    
#     submission = (
#         sample_prediction_df_copy
#         [['date_playerId']]
#         .reset_index()  #  preserve index 'date'
#         .merge(sample_prediction_df[['date_playerId','target1','target2','target3','target4']],
#                how='left', on='date_playerId')
#         .set_index('date')  #  restore index 'date'
#     )
    
#     del test_df
#     del players_curr_day
#     del test_ads_ext_final
#     #sample_prediction_df.drop(columns=['playerId'],inplace=True)
#     del sample_prediction_df
#     del sample_prediction_df_copy
    
#     print('--------------------------------- Submitting for counter :',counter,'------------------------------------')
#     counter = counter + 1
    
#     #print(sample_prediction_df[0:2])
    
#     env.predict(submission)
    
    
    

In [ ]:
# print(submission.dtypes)
# submission.head()
